In [1]:
import pandas as pd
import numpy as np
import ast

from dotenv import load_dotenv
from llm_requests import *
from pinecone_db import *

from rag import RAG

from tqdm import tqdm

load_dotenv()

Package protobuf is installed but has a version conflict:
	(protobuf 3.20.3 (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages), Requirement.parse('protobuf>=4.23.2'))

This package is optional for trulens_eval so this may not be a problem but if
you need to use the related optional features and find there are errors, you
will need to resolve the conflict:

    ```bash
    pip install 'protobuf>=4.23.2'
    ```

If you are running trulens_eval in a notebook, you may need to restart the
kernel after resolving the conflict. If your distribution is in a bad place
beyond this package, you may need to reinstall trulens_eval so that all of the
dependencies get installed and hopefully corrected:
    
    ```bash
    pip uninstall -y trulens_eval
    pip install trulens_eval
    ```

Package nbconvert is installed but has a version conflict:
	(nbconvert 6.3.0 (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages), Requirement.parse('nbconver

True

In [2]:
import os
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

os.environ['TOKENIZERS_PARALLELISM'] = "false"

In [3]:
def init_db():
    PINECONE_KEY = os.getenv('PINECONE_API_KEY')
    db = VectorDB(PINECONE_KEY)
    db.set_index()
    return db

DATASET = os.getenv('DATASET')
db = init_db()
rag = RAG(db)

data = pd.read_csv(DATASET)
data['genres'] = data['genres'].apply(ast.literal_eval)

In [3]:
from trulens_eval import Tru

tru = Tru()


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [5]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI

import numpy as np


provider = OpenAI()

grounded = Groundedness(groundedness_provider=provider)

# Groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Relevance between overall query and answer
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Relevance between search query and each context chunk
f_context_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


In [6]:
from trulens_eval import TruCustomApp
# TODO
# {embedding_model}-{generator_model}-{datetime}
tru_recorder = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_groundedness, f_answer_relevance, f_context_relevance])

In [7]:
def search_movies(db, query: str, metadata: dict, data: pd.DataFrame, k=10, min_similarity_score=0) -> pd.DataFrame:
    result = db.search_movies(query, metadata, k=k, min_similarity_score=min_similarity_score)
    titles = np.unique([movie['title'] for movie in result])
    titles, descriptions = [], []
    for movie in result:
        title, description = movie['title'], movie['description']
        if title not in titles:
            titles.append(title)
            descriptions.append(description)

    data = data[data['title'].isin(titles)]
    data['rag_description'] = None

    for title, description in zip(titles, descriptions):
        data.loc[data['title'] == title, 'rag_description'] = description

    return data


query = "An interesting comedy movie"
metadata = extract_metadata(query)
metadata = ast.literal_eval(metadata['generated_text'])

search_movies(db, query, metadata, data, k=5)

,title,genres,year,description,rag_description
107,Pie in the Sky,"[Comedy, Romance]",1996,A whimsical saga of a young man whose two life...,page_content='A whimsical saga of a young man ...
2786,Everything You Always Wanted to Know About Sex...,[Comedy],1972,Everything You Always Wanted to Know About Sex...,page_content='Critical response\nThe film hold...
3929,Comedian,"[Comedy, Documentary]",2002,Comedian is a 2002 American documentary film f...,page_content='Advertising\nThe film\'s unusual...
5956,Looking for Comedy in the Muslim World,[Comedy],2005,Looking for Comedy in the Muslim World is a 20...,page_content='Looking for Comedy in the Muslim...
8785,Ashby,"[Comedy, Drama]",2015,High-school nerd Ed Wallis (Nat Wolff) enters ...,"page_content=""High-school nerd Ed Wallis (Nat ..."


In [8]:
with open('TestQueries.txt', 'r', encoding='utf-8') as f:
    search_queries = f.readlines()
    search_queries = list(map(lambda q: q.strip(), search_queries))

search_queries[:3]

['\ufeffAnimation movie toys move into new house, jealous toy sheriff, space ranger action figure, sadistic neighbor, mutant toys, Pizza Planet, Christmas gift-opening scene',
 'Documentary martial arts dance rituals, war dances, sword dances, cultural significance, martial arts performance with music',
 'Dark comedy series based on graphic novel, teen psychopath road trip, rebel adventure, star-crossed teenagers']

In [9]:
q = 'Animation movie toys move into new house, jealous toy sheriff, space ranger action figure, sadistic neighbor, mutant toys, Pizza Planet, Christmas gift-opening scene'
rag.retrieve(q)

'Toy Story\npage_content="Sentient when humans are not around, a group of toys are preparing to move into a new house with their owner, child Andy Davis, his younger sister Molly and their single mother Mrs. Davis. Sheriff Woody, Andy\'s favorite toy and their de facto leader, sends Sarge and his green army men to spy on Andy\'s birthday party with a baby monitor, as the toys are nervous about him receiving new toys that could replace them. The other toys include Mr. Potato Head, Slinky Dog, Rex the tyrannosaur, Hamm the piggy bank, and Bo Peep the porcelain doll. Andy receives a Buzz Lightyear action figure who believes he is an actual Space Ranger and does not know he is a toy. Buzz impresses the others with his high-tech features and becomes Andy\'s new favorite toy, making Woody jealous."'

In [10]:
for query in tqdm(search_queries):
    success = False
    while not success:
        try:
            with tru_recorder as recording:
                rag.query(query)
            success = True
        except Exception:
            print(query)

Validation error: 1 validation error for Rating
rating
  Value error, Rating must be between 0 and 10 [type=value_error, input_value=17, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error
Error calling wrapped function generate_completion.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/instruments.py", line 652, in tru_wrapper
    rets, cost = Endpoint.track_all_costs_tally(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 492, in track_all_costs_tally
    result, cbs = Endpoint.track_all_costs(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 473, in track_all_costs
    return Endpoint._track_costs(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site

Movie street magician girl with ability to see ghosts, developing friendship, vengeful ghost best friend, romance


Error calling wrapped function retrieve.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/instruments.py", line 652, in tru_wrapper
    rets, cost = Endpoint.track_all_costs_tally(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 492, in track_all_costs_tally
    result, cbs = Endpoint.track_all_costs(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 473, in track_all_costs
    return Endpoint._track_costs(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 570, in _track_costs
    result: T = __func(*args, **kwargs)
  File "/Users/markiian_tsalyk/Desktop/UCU/4th_year/term2/diploma/DomainSpecificAIAssistant/evaluation/rag.py", line 2

Children's fantasy film about a group of kids who discover a hidden portal to a magical realm, encountering mythical creatures, solving riddles, and facing challenges to save their world


Error calling wrapped function retrieve.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/instruments.py", line 652, in tru_wrapper
    rets, cost = Endpoint.track_all_costs_tally(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 492, in track_all_costs_tally
    result, cbs = Endpoint.track_all_costs(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 473, in track_all_costs
    return Endpoint._track_costs(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 570, in _track_costs
    result: T = __func(*args, **kwargs)
  File "/Users/markiian_tsalyk/Desktop/UCU/4th_year/term2/diploma/DomainSpecificAIAssistant/evaluation/rag.py", line 2

...

Timeless classic movie from the golden age of Hollywood, featuring iconic performances, memorable dialogue, and a plot that has stood the test of time


Error calling wrapped function generate_completion.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/instruments.py", line 652, in tru_wrapper
    rets, cost = Endpoint.track_all_costs_tally(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 492, in track_all_costs_tally
    result, cbs = Endpoint.track_all_costs(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 473, in track_all_costs
    return Endpoint._track_costs(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 570, in _track_costs
    result: T = __func(*args, **kwargs)
  File "/Users/markiian_tsalyk/Desktop/UCU/4th_year/term2/diploma/DomainSpecificAIAssistant/evaluation/rag.

Timeless classic movie from the golden age of Hollywood, featuring iconic performances, memorable dialogue, and a plot that has stood the test of time


Error calling wrapped function retrieve.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/instruments.py", line 652, in tru_wrapper
    rets, cost = Endpoint.track_all_costs_tally(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 492, in track_all_costs_tally
    result, cbs = Endpoint.track_all_costs(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 473, in track_all_costs
    return Endpoint._track_costs(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 570, in _track_costs
    result: T = __func(*args, **kwargs)
  File "/Users/markiian_tsalyk/Desktop/UCU/4th_year/term2/diploma/DomainSpecificAIAssistant/evaluation/rag.py", line 2

Timeless classic movie from the golden age of Hollywood, featuring iconic performances, memorable dialogue, and a plot that has stood the test of time


In [11]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,RAG v1,"{""tru_class_info"": {""name"": ""TruCustomApp"", ""m...",RAG(rag),record_hash_b277b80681edcfbdcd189703bb99a8df,"""\ufeffAnimation movie toys move into new hous...","""Toy Story is relevant for you because it cont...",-,"{""record_id"": ""record_hash_b277b80681edcfbdcd1...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-17T14:57:14.149335"", ""...",2024-03-17T14:57:44.781328,1.0,1.0,0.385714,[{'args': {'prompt': '﻿Animation movie toys mo...,[{'args': {'question': '﻿Animation movie toys ...,[{'args': {'source': ['Toy Story\npage_content...,30,0,0.0
1,RAG v1,"{""tru_class_info"": {""name"": ""TruCustomApp"", ""m...",RAG(rag),record_hash_8f1ca19d5e73a7457344ba594a54af45,"""Documentary martial arts dance rituals, war d...","""War Dance is a documentary that explores the ...",-,"{""record_id"": ""record_hash_8f1ca19d5e73a745734...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-17T14:57:45.269637"", ""...",2024-03-17T14:58:13.354595,1.0,0.8,0.560000,[{'args': {'prompt': 'Documentary martial arts...,[{'args': {'question': 'Documentary martial ar...,"[{'args': {'source': [""War Dance\npage_content...",28,0,0.0
2,RAG v1,"{""tru_class_info"": {""name"": ""TruCustomApp"", ""m...",RAG(rag),record_hash_f03ebda9402b3f2dbd1e071d7a8dd49c,"""Dark comedy series based on graphic novel, te...","""The End of the F***ing World is relevant for ...",-,"{""record_id"": ""record_hash_f03ebda9402b3f2dbd1...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-17T14:58:13.684338"", ""...",2024-03-17T14:58:37.227926,1.0,1.0,0.675000,[{'args': {'prompt': 'Dark comedy series based...,[{'args': {'question': 'Dark comedy series bas...,"[{'args': {'source': [""The End of the F***ing ...",23,0,0.0
3,RAG v1,"{""tru_class_info"": {""name"": ""TruCustomApp"", ""m...",RAG(rag),record_hash_3564be3a181e0185f9337f40e72d7a6c,"""TV series rich kid high school president ambi...","""The Politician is relevant for you because it...",-,"{""record_id"": ""record_hash_3564be3a181e0185f93...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-17T14:58:37.561130"", ""...",2024-03-17T14:58:59.673074,0.9,1.0,0.250000,[{'args': {'prompt': 'TV series rich kid high ...,[{'args': {'question': 'TV series rich kid hig...,[{'args': {'source': ['The Politician\npage_co...,22,0,0.0
4,RAG v1,"{""tru_class_info"": {""name"": ""TruCustomApp"", ""m...",RAG(rag),record_hash_5ced51e40c094c587fc86a448f2c4e3c,"""Movie Upper Paleolithic Europe hunting expedi...","""\nRELEVANCE REASONING OF WHITE FANG MOVIE FOR...",-,"{""record_id"": ""record_hash_5ced51e40c094c587fc...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-17T14:58:59.975280"", ""...",2024-03-17T14:59:31.032604,1.0,0.4,0.000000,[{'args': {'prompt': 'Movie Upper Paleolithic ...,[{'args': {'question': 'Movie Upper Paleolithi...,[{'args': {'source': ['White Fang\npage_conten...,31,0,0.0


In [15]:
pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Groundedness,Context Relevance,Answer Relevance
0,"""\ufeffAnimation movie toys move into new house, jealous toy sheriff, space ranger action figure, sadistic neighbor, mutant toys, Pizza Planet, Christmas gift-opening scene""","""Toy Story is relevant for you because it contains the following elements from your query: \n1. Animation movie - Toy Story is an animated film that features a group of sentient toys who come to life when humans are not around.\n2. Toys move into new house - The story follows the toys as they prepare to move into a new house with their owner Andy and his family. \n3. Jealous toy sheriff - Woody, the de facto leader of the toys, becomes jealous when Andy starts to favor Buzz Lightyear, a new action figure that he believes will replace him as Andy's favorite toy. \n4. Space ranger action figure - Buzz Lightyear is an action figure who believes he is an actual space ranger and not just a toy.\n5. Sadistic neighbor - The toys encounter a sadistic next-door neighbor named Sid, who enjoys torturing and disassembling his own toys. \n6. Mutant toys - Some of the toys in the film have mutated or been modified in some way, such as Mr. Potato Head's detachable parts and Slinky Dog""",0.385714,1.0,1.0
1,"""Documentary martial arts dance rituals, war dances, sword dances, cultural significance, martial arts performance with music""","""War Dance is a documentary that explores the martial arts dance rituals and war dances from various cultures around the world. The film showcases the cultural significance of these performances and how they are used to celebrate valor and conquest. The movie features different forms of weapon dance, including sword dances, and highlights the unique aspects of each tradition.\nThe film also delves into the martial arts performance with music, exploring the strong percussive rhythms that accompany these performances. It provides a comprehensive look at the cultural significance of war dances and their connection to martial arts, making it relevant for viewers interested in documentaries about dance, culture, and martial arts.""",0.560000,0.8,1.0
2,"""Dark comedy series based on graphic novel, teen psychopath road trip, rebel adventure, star-crossed teenagers""","""The End of the F***ing World is relevant for you because it features all the elements mentioned in your query. It's a darkly comedic series that follows two teenagers, one with psychopathic tendencies and the other a rebellious adventurer, as they embark on a star-crossed road trip. The show explores themes of identity, morality, and the complexities of human relationships, making it an engaging and thought-provoking watch.""",0.675000,1.0,1.0
3,"""TV series rich kid high school president ambition, treacherous political landscape, high school politics""","""The Politician is relevant for you because it deals with themes of wealth and power in a high school setting. The main character Payton has grand ambitions to become the President of the United States one day, but first he must navigate the cutthroat world of high school politics. This show explores how far people will go to achieve their goals, even if it means betraying others or sacrificing their own morals. The treacherous political landscape in this show is not just limited to the high school setting, but also reflects on the current state of American politics.""",0.250000,1.0,0.9
4,"""Movie Upper Paleolithic Europe hunting expedition, tribe chief's son survival journey, bonding with wolf, coming of age story, Alpha wolf movie""","""\nRELEVANCE REASONING OF WHITE FANG MOVIE FOR THE USER QUERY: \nThe movie White Fang is relevant to the user query because it features a hunting expedition in a remote and harsh environment, specifically Upper Paleolithic Europe. The story follows the journey of the chief's son as he faces challenges and learns important lessons about survival, bonding with a wolf, and coming of age. The Alpha wolf in the movie also aligns w

In [13]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
RAG v1,0.33273,0.572414,0.858621,28.566667,0.0


In [4]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.0.139:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>